In [1]:
import tensorflow.keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import model_from_json
import numpy as np
import pandas as pd
import re, pickle

In [2]:
# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model.h5")

print("Loaded model from disk")

Loaded model from disk


In [3]:
tweets = pd.read_csv('./Tweets.csv',sep=',')

data = tweets[['text','airline_sentiment']]

data = data[data.airline_sentiment != "neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('(@[A-Za-z0-9]+)|(\w+:\/\/\S+)|([^a-zA-z0-9\s])','',x)))

print(data[ data['airline_sentiment'] == 'positive'].size)
print(data[ data['airline_sentiment'] == 'negative'].size)

X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X, maxlen=40)

data.loc[data['airline_sentiment']=='positive', 'airline_sentiment'] = 4
data.loc[data['airline_sentiment']=='negative', 'airline_sentiment'] = 0

4726
18356


In [ ]:
Y = pd.get_dummies(data['airline_sentiment']).values

pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X)):
    result = loaded_model.predict(X[x].reshape(1,X.shape[1]),batch_size=1,verbose = 2)[0]
    if np.argmax(result) == np.argmax(Y[x]):
        if np.argmax(Y[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
    if np.argmax(Y[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1
    if x%1000 == 0:
        print(x, 'done')

print()
print("pos_acc=", pos_correct/pos_cnt*100, "%")
print("neg_acc=", neg_correct/neg_cnt*100, "%")

0 done
1000 done
2000 done
3000 done
4000 done
5000 done
6000 done
7000 done
8000 done
9000 done
10000 done
